# Cinnamon A Framework For Scale Out Encrypted AI
## Notebook 1: Getting Started

Welcome to the Part 1 of the tutorial on the Cinnamon Framework. The Cinnamon Framework consists of four components:

1. The Cinnamon DSL and Compiler
1. The Cinnamon Compiler
3. The Cinnamon Emulator
4. The Cinnamon Accelerator Simulator

![image](images/Overview.jpg)

Part 1 of the tutorial covers writing programs in the Cinnamon DSL, compiling them and testing the compiled outputs with the emulator. Part 2 of the tutorial looks into using the parallelization techniques and compiler optimizations presented in the Cinnamon paper and simulating them on the accelerator simulator. 

Author:
- Siddharth Jayashankar (sidjay@cmu.edu)

## Homomorphic Encryption
Homomorphic Encryption (HE) is an advanced cryptographic technique that enables direct computation on encrypted data. There are two types of HE - Partial and Full. Partially Homomorphic Encryption enables a limited number of encrypted computations. Fully Homomorphic Encryption, on the other hand, enables unlimited computation on encrypted data.


## In This Tutorial
In this tutorial, we will be using the CKKS scheme at a 128 bit security level. CKKS is an FHE scheme that supports encrypted computation over real numbers. This makes it suitable for encrypted AI applications.

## Exercise 1: Writing an FHE Program To Add 2 Encrypted Values
In this exercise, we will use the Cinnamon framework to write and run an FHE program to add two encrypted numbers together.

![image](images/Hadd.jpg)

### Exercise 1.1: Writing the Program in the Cinnamon DSL

The Cinnamon Compiler is a python embedded DSL. Let's import the Cinnamon modules

In [1]:
## Import the cinnamon modules
from cinnamon.dsl import * # Import the Cinnamon DSL
from cinnamon.compiler import * # Import the Cinnamon Compiler

Now let's write a simple program to add two numbers. We first create a `CinnamonProgram` object. The first argument to the constructor is the name of the program. If you don't understand the other arguments, don't worry, these will be made clear later on. 

In [2]:
# Create a Cinnamon Program
TOP_LEVEL=51
add_program = CinnamonProgram(name='Add',rns_bit_size=28,num_chips=1)

Now, we need to create two ciphertexts in the program and add them together. The constructor requires information about two attributes of the ciphertext: scale and level. We will cover these concepts in a later part of the tutorial.

The Cinnamon compiler uses the information about the scale and levels provided here to type check the program. If you add values with incompatible scales and levels, the compiler will produce an error.

In [3]:
with add_program:
    level = TOP_LEVEL
    scale = 56 # scale in bits

    # Create a CiphertextInput named "x" with scale and level attribute values
    x = CiphertextInput("x",scale,level)
    # TODO Create a CiphertextInput y with the same scale and level as x
    y = 
    # TODO: Assign z as the sum of x and y
    z = 
    # Set the variable z to be an output of the program named 'z'
    Output("z",z) 

### Exercise 1.2 Compiling the FHE program using the Cinnamon Compiler
Now, lets use the Cinnamon compiler to compiler our program down to the Cinnamon ISA. 

First, lets create a folder for the outputs of the compilation

In [4]:
add_program_dir = "add_program_outputs"
!mkdir -p "{add_program_dir}"

Now let's use the Cinnamon compiler to compile our program.

In [ ]:
cinnamon_compile(add_program,TOP_LEVEL,1,1024,f"{add_program_dir}/")

Once the compilation is done, you will be able to see the outputs of the compilation in the `add_program_dir` directory. There will be two kinds of outputs: inputs and instructions. The inputs file contains the mapping from user specified symbols for the inputs and outputs and the instructions file contains the actual instructions in the Cinnamon assembly.

### Exercise 1.3: Emulating The Program
Now that we have compiled the program, we want to test our program and the compiler output. To do so, we use the Cinnamon Emulator. The Cinnamon Emulator reads in Cinnamon assembly and emulates the instructions on a CPU. But before, we do that, we need to actually assign values and scale information to our program inputs and outputs.

First, let's import the cinnamon emulator

In [6]:
import cinnamon_emulator

We need to define some parameters here.
`SLOTS=32*1024` and `RNS_PRIMES`. `SLOTS` is the number of plaintext values each ciphertext object holds. This is usually a power of 2. For CKKS, a slot size of 32K is typically used. All homomorphic operations take place in a SIMD over the slots.

`RNS_PRIMES` is a list of primes that we intend to use as the basis in the [Residue Number System(RNS)](). CKKS ciphertexts are composed of elements from a modular ring. The modulus is typically a very large integer, on the order of 1000s of bits. Arithmetic over such large integers is very slow. However, if the ciphertext modulus is carefully chosen as a product of a list of special primes, the Reisude Number System (RNS) can be used to speedup the artimetic. `RNS_PRIMES` correspond to this list of primes. 

Since the Cinnamon archtecture is designed to work with 28bit primes, we select all our RNS Primes to be 28bit values. 
The ratio of `SLOTS` to the product of the `RNS_PRIMES` determines the security of the scheme. We select values for a 128 bit security level.

In [7]:
# These parameters are for 128 bit security
SLOTS = 32*1024
RNS_PRIMES = [204865537, 205651969, 206307329, 207880193, 209059841, 210370561, 211025921, 211812353, 214171649, 215482369, 215744513, 216137729, 216924161, 217317377, 218628097, 219676673, 220594177, 221249537, 222035969, 222167041, 222953473, 223215617, 224002049, 224133121, 225574913, 228065281, 228458497, 228720641, 230424577, 230686721, 230817793, 231473153, 232390657, 232652801, 234356737, 235798529, 236584961, 236716033, 239337473, 239861761, 240648193, 241827841, 244842497, 244973569, 245235713, 245760001, 246415361, 249561089, 253100033, 253493249, 254279681, 256376833, 256770049, 257949697, 258605057, 260571137, 260702209, 261488641, 261881857, 263323649, 263454721, 264634369, 265420801, 268042241]

Now lets create our inputs and output specifications. Input values must be vectors of `SLOTS` values.

In [8]:
add_program_inputs = {}
add_program_output_scales = {}

import numpy as np
x = np.array([i % 64 for i in range(SLOTS)])
# TODO: Initialize y to an array of size SLOTS
y = np.array()

SCALE = 1 << 56
add_program_inputs["x"] = (x,SCALE)
# TODO: Set the scales for inputs y and output z
add_program_inputs["y"] = (,)
add_program_output_scales["z"] = 

Let's create a secret key to encrypt and decrypt our values. For 128 bit security, we generate a ternary key with a hamming weight of `SLOTS`.

In [9]:
import random
random.seed(10)
def generate_secret_key(Slots,HammingWeight=32):
    secretKey = [0]*(2*Slots)
    count = 0
    while count < HammingWeight:
        pos = random.randint(0,2*Slots-1)
        if secretKey[pos] != 0:
            continue
        val = random.randint(0,1)
        if val == 0:
            secretKey[pos] = -1
        elif val == 1:
            secretKey[pos] = 1
        else:
            raise Exception("")
        count += 1
    return secretKey

secretKey = generate_secret_key(SLOTS,HammingWeight=SLOTS)

Now let's create the ciphertexts and run the encrypted program. First we create a context object with information about `SLOTS` and `RNS_PRIMES`.

In [10]:
context = cinnamon_emulator.Context(SLOTS,RNS_PRIMES)

Then we create an encryptor with information about our secret key to perform the encryption.

In [11]:
encryptor = cinnamon_emulator.CKKSEncryptor(context,secretKey)

Finnaly, we create an emulator object.

In [12]:
emulator = cinnamon_emulator.Emulator(context)

Now let's create the ciphertext and plaintext inputs to the program.

In [ ]:

emulator.generate_and_serialize_evalkeys(f"{add_program_dir}/evalkeys",f"{add_program_dir}/program_inputs",encryptor)
emulator.generate_inputs(f"{add_program_dir}/program_inputs",f"{add_program_dir}/evalkeys",add_program_inputs,encryptor)

In [ ]:

emulator.run_program(f"{add_program_dir}/instructions",1,1024)

Once the program completes, we can get the decrypt the outputs and return them. The actual output will be an array of size `SLOTS`. For brevity, let's print the first 10 values.

In [ ]:
outputs = emulator.get_decrypted_outputs(encryptor,add_program_output_scales)
decrypted_z = outputs["z"]
print("Decrypted z: ",decrypted_z[0:10])

You might have noticed that the output of the program is actually an array of complex numbers. This is because the CKKS scheme actually operates over complex numbers and not real numbers. 

Let's convert the values to real values by dropping the complex part and compare them to the expected output.

In [ ]:
decrypted_z_real = [i.real for i in decrypted_z]
z = x+y
mse = np.mean(np.square(decrypted_z_real - z))
print(f"Encrypted Output: {decrypted_z_real[0:10]}")
print(f"Expected Ouptut:  {z[0:10]}")
print(f"Mean Square Error: {mse}")

You will notice that the values in the encrypted output are approximate. The precision of the approximation is controlled by the `SCALE` we set when creating the inputs. I encourage you to experiment with different values of `SCALE` to see how it changes the precision of the output.

Congratulations! You have just written your first program in the Cinnamon framework and completed Exercise 1 of the tutorial.

## Exercise 2: Multiplication and Rotation of Encrypted Values
In this exercise, we will implement the operations of multiplication (*) and rotation (<<) on ciphertexts in Cinnamon

### Exercise 2.1 Writing and Compiling the Cinnamon Program

Similar to Exercise 1.1, let's create a Cinnamon Program named `multiply_and_rotate` with two CiphertextInputs x and y. We multiply the ciphertexts using the inplace operator `*`. After multiplying two ciphertexts, we call the `.relinearize()` method to relinearize the Ciphertexts. Relinearization is required to bring the ciphertexts back to a usable form for further computation after a multiplication. Next, we left rotate the ciphertext using the `<<` inplace operator and specifying the rotation step size (2 in this example).

Finally, we need to rescale the Ciphertexts. This is because multiplcation of two operands also multiplies their scales. In the Cinnamon DSL, you can get the scale of any object using the `.scale()` method. 

Rescaling of ciphertexts is done through the `.rescale()` method. Rescaling divides the scale of the ciphertext by the `RNS_PRIME` corresponding to the level. Since we use 28bit RNS_PRIMES, we set the rns_bit_size argument of the `CinnamonProgram` to 28 in this example. Thus, a call to `.rescale()` will reduce the scale by 28.

In [ ]:
multiply_rotate_program = CinnamonProgram("multiply_and_rotate",rns_bit_size=28,num_chips=1)
with multiply_rotate_program:
    level = TOP_LEVEL
    scale = 56 # scale in bits
    x = CiphertextInput("x",scale,level)
    y = CiphertextInput("y",scale,level)
    # TODO: Set z to be product of x and y
    z = 
    # TODO: Relinearize z
    z = 
    # TODO: Left rotate z by 2
    z = 
    print(f"z before rescale: scale: {z.scale()} level: {z.level()}")
    z = z.rescale().rescale()
    print(f"z after rescale:  scale: {z.scale()}  level: {z.level()}")
    Output("z",z)

Now let's compile the program

In [ ]:
mul_rot_program_dir = "mul_rot_program_outputs"
!mkdir -p "{mul_rot_program_dir}"
cinnamon_compile(multiply_rotate_program,TOP_LEVEL,1,1024,f"{mul_rot_program_dir}/")

### Exercise 2.2: Emulating The Program

In [19]:
# These parameters are for 128 bit security
SLOTS = 32*1024
RNS_PRIMES = [204865537, 205651969, 206307329, 207880193, 209059841, 210370561, 211025921, 211812353, 214171649, 215482369, 215744513, 216137729, 216924161, 217317377, 218628097, 219676673, 220594177, 221249537, 222035969, 222167041, 222953473, 223215617, 224002049, 224133121, 225574913, 228065281, 228458497, 228720641, 230424577, 230686721, 230817793, 231473153, 232390657, 232652801, 234356737, 235798529, 236584961, 236716033, 239337473, 239861761, 240648193, 241827841, 244842497, 244973569, 245235713, 245760001, 246415361, 249561089, 253100033, 253493249, 254279681, 256376833, 256770049, 257949697, 258605057, 260571137, 260702209, 261488641, 261881857, 263323649, 263454721, 264634369, 265420801, 268042241]

In [20]:
mul_rot_program_inputs = {}
mul_rot_program_output_scales = {}

import numpy as np
x = np.array([i % 64 for i in range(SLOTS)])
# TODO: Initialize y to  an array of size SLOTS
y = np.array()

SCALE = 1 << 56
level = TOP_LEVEL
mul_rot_program_inputs["x"] = (x,SCALE)
mul_rot_program_inputs["y"] = (y,SCALE)
z_scale_before_rescale = SCALE * SCALE
z_scale_after_rescale = z_scale_before_rescale / (RNS_PRIMES[level -1]*RNS_PRIMES[level - 2])
mul_rot_program_output_scales["z"] = z_scale_after_rescale

In [ ]:
context = cinnamon_emulator.Context(SLOTS,RNS_PRIMES)
encryptor = cinnamon_emulator.CKKSEncryptor(context,secretKey)
emulator = cinnamon_emulator.Emulator(context)

base_dir = mul_rot_program_dir
emulator.generate_and_serialize_evalkeys(f"{base_dir}/evalkeys",f"{base_dir}/program_inputs",encryptor)
emulator.generate_inputs(f"{base_dir}/program_inputs",f"{base_dir}/evalkeys",mul_rot_program_inputs,encryptor)
emulator.run_program(f"{base_dir}/instructions",1,1024)
emulator_outputs = emulator.get_decrypted_outputs(encryptor,mul_rot_program_output_scales)

In [ ]:
decrypted_z = emulator_outputs["z"]
decrypted_z_real = [i.real for i in decrypted_z]

def rotate(x,n):
    return np.concat((x[n:], x[:n]))
z = rotate(x * y,2)
mse = np.mean(np.square(decrypted_z_real - z))
print(f"Encrypted Output: {decrypted_z_real[0:10]}")
print(f"Expected Ouptut:  {z[0:10]}")
print(f"Mean Square Error: {mse}")

Congratulations! This marks the end of notebook1. In [notebook2](../notebook2/notebook2.ipynb), we will write and run an actual encrypted AI inference using Cinnamon.